## Generating PV curves from a Beta PDF

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
# Data irradiance taken from Nrel, this numbers represents the geolocalization
data = pd.read_csv(r'../dataset\NSRDB_Irradiance_Data_Portugal_109749_41.13_-8.62_2017.csv', header=2)
# GHI is a mesurament of irradiance
ghi = data['GHI'].to_numpy(dtype=float)
# Normalizing this measurament to kW/m2
ghi = ghi/1000
# Also collecting the temperature
temp = data['Temperature'].to_numpy(dtype=float)


In [13]:
# Separating the irradiance data from january and july that represents Winter and Summer
irrad_jan = ghi[:2976]
irrad_jul = ghi[17376:20352]

# Choosing at random 7 days (of 96 timesteps) from the dataset
irrad_jan = [irrad_jan[576:672], irrad_jan[1440:1536], irrad_jan[1536:1632], irrad_jan[1632:1728], 
            irrad_jan[1728:1824], irrad_jan[1824:1920], irrad_jan[1920:2016]]
irrad_jul = [irrad_jul[:96], irrad_jul[96:192], irrad_jul[192:288], irrad_jul[288:384],
            irrad_jul[960:1056], irrad_jul[1056:1152], irrad_jul[1152:1248], irrad_jul[1440:1536],
            irrad_jul[1920:2016], irrad_jul[2112:2208], irrad_jul[2400:2496],  irrad_jul[2496:2592]]

# Calculating the mean and standard deviation
irrad_medjan = np.mean(irrad_jan, 0)
irrad_medjul = np.mean(irrad_jul, 0)
irrad_stdjan = np.std(irrad_jan, 0)  
irrad_stdjul = np.std(irrad_jul, 0)

In [14]:
# Separating the temperature data from january and july that represents Winter and Summer
temp_jan = temp[:2976]
temp_jul = temp[17376:20352]

# Choosing at random 7 days (of 96 timesteps) from the dataset
temp_jan = [temp_jan[576:672], temp_jan[1440:1536], temp_jan[1536:1632], temp_jan[1632:1728], 
            temp_jan[1728:1824], temp_jan[1824:1920], temp_jan[1920:2016]]
temp_jul = [temp_jul[:96], temp_jul[96:192], temp_jul[192:288], temp_jul[288:384],
            temp_jul[960:1056], temp_jul[1056:1152], temp_jul[1152:1248], temp_jul[1440:1536],
            temp_jul[1920:2016], temp_jul[2112:2208], temp_jul[2400:2496],  temp_jul[2496:2592]]

# Calculating the mean
temp_medjan = np.mean(temp_jan, 0)
temp_medjul = np.mean(temp_jul, 0)


### Beta PDF

It is used this formulation from https://en.wikipedia.org/wiki/Beta_distribution to generate the curves

![Beta PDF](../Images/beta_pdf.png)

### Calculating alfa and beta for january and july

In [15]:
# Calculating alfa and beta that represents one day of January
beta_jan = np.zeros((np.shape(irrad_medjan)))
alpha_jan = np.zeros((np.shape(irrad_medjan)))
for i in range(96):
    if irrad_stdjan[i] == 0:
        beta_jan[i] = 0
        alpha_jan[i] = 0
    else:
        beta_jan[i] = (1-irrad_medjan[i])*((irrad_medjan[i]*(1-irrad_medjan[i]))/(irrad_stdjan[i]**2) - 1)
        alpha_jan[i] = (irrad_medjan[i]*beta_jan[i])/(1-irrad_medjan[i])

# Calculating alfa and beta that represents one day of July
beta_jul = np.zeros((np.shape(irrad_medjul)))
alpha_jul = np.zeros((np.shape(irrad_medjul)))
for i in range(96):
    if irrad_stdjul[i] == 0:
        beta_jul[i] = 0
        alpha_jul[i] = 0
    else:
        beta_jul[i] = (1-irrad_medjul[i])*((irrad_medjul[i]*(1-irrad_medjul[i]))/(irrad_stdjul[i]**2) - 1)
        alpha_jul[i] = (irrad_medjul[i]*beta_jul[i])/(1-irrad_medjul[i])

In [16]:
alpha_jan

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  73.25660256, 154.88136514, 339.87678834,
       482.20967237, 499.92511767, 511.3328    , 491.65905963,
       437.36686476, 424.98389645, 430.18911   , 415.3780801 ,
       409.14580175, 453.39991898, 496.06265677, 619.96423077,
       655.0686968 , 622.83976049, 566.59410716, 519.45849286,
       495.55645427, 465.58223643, 436.42106824, 391.4925793 ,
       362.39325118, 344.00344622, 328.93637657, 297.9989755 ,
       271.38577222, 260.10036197, 233.29904566, 206.53

### Generate the irradiance curves for one day, using Beta PDF

In [17]:
# Number of samples
N = 1000 

# Initialize a random generator
rng = np.random.default_rng(15)

# Initialize the arrays of irradiance curves
irrad_winter = np.zeros((N, 96))
irrad_summer = np.zeros((N, 96))

# Generate N samples of 96 timesteps (1 day)
for i in range(96):
    if (alpha_jan[i] or beta_jan[i]) <= 0:
        irrad_winter[:,i] = 0
    else:
        irrad_winter[:,i] = rng.beta(alpha_jan[i], beta_jan[i], N) 

for i in range(96):
    if (alpha_jul[i] or beta_jul[i]) <= 0:
        irrad_summer[:,i] = 0
    else:
        irrad_summer[:,i] = rng.beta(alpha_jul[i], beta_jul[i], N)

### Converting this irradiance into Power generated by the PV system 

In [9]:
# Data of the PV system 
pmpref = 780  # in (Wp)
noct   = 45   # Nominal operating cell temperature (Datasheet)
n      = 220   # Number of modules 
ninv   = 0.95 # Inverter efficiency

# From kW/m2 to W/m2
irrad_winter = irrad_winter*1000
irrad_summer = irrad_summer*1000

# Initializing arrays
tc_w = np.zeros((N, 96))
tc_s = np.zeros((N, 96))
pmp_w = np.zeros((N, 96))
pmp_s = np.zeros((N, 96))
ppv_w = np.zeros((N, 96))
ppv_s = np.zeros((N, 96))

# Calculating the power generated by the PVs
for i in range(N):
    for j in range(96):
        tc_w[i,j] = temp_medjan[j]+((noct-20)/800)*irrad_winter[i,j]
        tc_s[i,j] = temp_medjul[j]+((noct-20)/800)*irrad_summer[i,j]
        pmp_w[i,j] = n*pmpref*(irrad_winter[i,j]/1000)*(1-0.0043*(tc_w[i,j]-25))
        pmp_s[i,j] = n*pmpref*(irrad_summer[i,j]/1000)*(1-0.0043*(tc_s[i,j]-25)) 
        ppv_w[i,j] = (pmp_w[i,j]*ninv)*10**-3 # Potencia de saída da usina Fotovoltaica (em kW)
        ppv_s[i,j] = (pmp_s[i,j]*ninv)*10**-3

### Creating 144 pts from 96

In [10]:
# Arrays to store the 144pts curves
PV_s = np.zeros((N, 144))
PV_w = np.zeros((N, 144))
k = 1
l = 1
ll = 0
for mc in range(N):
   for i in range(2,96,2):
      PV_s[mc][i+k] = ppv_s[mc,i]
      PV_w[mc][i+k] = ppv_w[mc,i]
      k = k + 1
      PV_s[mc][i+ll] = (0.5*ppv_s[mc,i]+ppv_s[mc,i-1])/1.5 
      PV_w[mc][i+ll] = (0.5*ppv_w[mc,i]+ppv_w[mc,i-1])/1.5
      ll = ll + 1
    
   for j in range(3,96,2):
      PV_s[mc][j+l] = (2*ppv_s[mc,j] + ppv_s[mc,j-1])/3
      PV_w[mc][j+l] = (2*ppv_w[mc,j] + ppv_w[mc,j-1])/3
      l = l + 1
    
   k = 1 
   l = 1 
   ll = 0


# From kW to MW
PV_s = PV_s/1000
PV_w = PV_w/1000

date = datetime.now().strftime('%d_%m-%H_%M')
# Save the fotovoltaic curves
np.savez(f"../dataset/PV_Curves_{N}_Samples_{date}", PV_s=PV_s, PV_w=PV_w)